In [2]:
from pynq import Overlay
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import time
import cv2

IMG_ROWS            = 360
IMG_COLS            = 640

ol = Overlay("/home/xilinx/pynq/overlays/hls_test/convolution/convolution_backup.bit")
ol.download()
print(ol.ip_dict)
xlnk = Xlnk()
dma_ip = ol.axi_dma_0
dma_addr = ol.ip_dict["axi_dma_0"]["phys_addr"]

CTRL_BASE_ADDR  = 0x43C00000
CTRL_ADDR_RANGE = 0x10000
ctrl_bus_mmio = MMIO(CTRL_BASE_ADDR, CTRL_ADDR_RANGE)

########################
#convolution
########################
image = cv2.imread("000001.jpg")
gray_image_temp = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_image = cv2.resize(gray_image_temp,(IMG_COLS, IMG_ROWS), interpolation = cv2.INTER_AREA )
im_data = np.array(gray_image)
im_data_flat = im_data.flatten()

N = IMG_ROWS*IMG_COLS
input_buffer = xlnk.cma_array(shape=(N,), dtype=np.float32)
output_buffer = xlnk.cma_array(shape=((N),), dtype=np.float32)

for i in range(N) :
    input_buffer[i] = im_data_flat[i]
    
    
timer_start = time.perf_counter();
ctrl_bus_mmio.write(0,1) #start the convolution kernel
dma_ip.sendchannel.transfer(input_buffer)
dma_ip.recvchannel.transfer(output_buffer)
dma_ip.sendchannel.wait()
dma_ip.recvchannel.wait()
timer_end = time.perf_counter();
conv_time = timer_end - timer_start
print("Convolution time {:.12f} seconds".format(conv_time))
while( (ctrl_bus_mmio.read(0) & 2) == 0) :  #check if output is ready
    print("waiting")
print(output_buffer)

out_arry = np.empty(shape=(IMG_ROWS,IMG_COLS),dtype=np.float32)
for i in range(IMG_ROWS):
    for j in range(IMG_COLS):
        out_arry[i][j] = output_buffer[IMG_COLS*i + j]
cv2.imwrite("output_img.jpg", out_arry)

#for i in range(IMG_ROWS):
#    for j in range(IMG_COLS):
#        out_arry[i][j] = output_buffer[(IMG_ROWS*IMG_COLS) + IMG_COLS*i + j]
#cv2.imwrite("output_img_2.jpg", out_arry)

{'axi_dma_0': {'phys_addr': 1077936128, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_dma:7.1', 'state': None, 'interrupts': {'mm2s_introut': {'controller': 'axi_intc_0', 'index': 0, 'fullpath': 'axi_dma_0/mm2s_introut'}, 's2mm_introut': {'controller': 'axi_intc_0', 'index': 1, 'fullpath': 'axi_dma_0/s2mm_introut'}}, 'gpio': {}, 'fullpath': 'axi_dma_0', 'driver': <class 'pynq.lib.dma.DMA'>}, 'axi_intc_0': {'phys_addr': 1098907648, 'addr_range': 65536, 'type': 'xilinx.com:ip:axi_intc:4.1', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'axi_intc_0', 'driver': <class 'pynq.overlay.DefaultIP'>}, 'img_convolution_0': {'phys_addr': 1136656384, 'addr_range': 65536, 'type': 'xilinx.com:hls:img_convolution:1.0', 'state': None, 'interrupts': {}, 'gpio': {}, 'fullpath': 'img_convolution_0', 'driver': <class 'pynq.overlay.DefaultIP'>}}
Convolution time 0.241963469000 seconds
[ 0.  0.  0. ...,  0.  0.  0.]


True

0


1996
